**기본작업**

In [79]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

ref                                                                                     deadline             category                reward  teamCount  userHasEntered  
--------------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2024                                      2024-11-10 23:59:00  Featured         1,100,000 Usd        621           False  
https://www.kaggle.com/competitions/lmsys-chatbot-arena                                 2024-08-05 23:59:00  Research           100,000 Usd       1747           False  
https://www.kaggle.com/competitions/isic-2024-challenge                                 2024-09-06 23:59:00  Research            80,000 Usd       1460           False  
https://www.kaggle.com/competitions/rsna-2024-lumbar-spine-degenerative-classification  2024-10-08 23:59:00  Featured            50,000 Usd        892     

In [ ]:
!kaggle competitions download -c acquire-valued-shoppers-challenge

100% 2.85G/2.86G [00:17<00:00, 145MB/s]
100% 2.86G/2.86G [00:17<00:00, 172MB/s]


In [ ]:
!unzip acquire-valued-shoppers-challenge

Archive:  acquire-valued-shoppers-challenge.zip
  inflating: offers.csv.gz           
  inflating: sampleSubmission.csv.gz  
  inflating: testHistory.csv.gz      
  inflating: trainHistory.csv.gz     
  inflating: transactions.csv.gz     


**파일전처리**

In [ ]:
df=pd.read_csv('/content/offers.csv.gz')
his=pd.read_csv('/content/trainHistory.csv.gz')

In [ ]:
tr=pd.read_csv('/content/transactions.csv.gz',compression='gzip')

In [ ]:
df.head()

,offer,category,quantity,company,offervalue,brand
0,1190530,9115,1,108500080,5.00,93904
1,1194044,9909,1,107127979,1.00,6732
2,1197502,3203,1,106414464,0.75,13474
3,1198271,5558,1,107120272,1.50,5072
4,1198272,5558,1,107120272,1.50,5072


In [ ]:
his.head()

,id,chain,offer,market,repeattrips,repeater,offerdate
0,86246,205,1208251,34,5,t,2013-04-24
1,86252,205,1197502,34,16,t,2013-03-27
2,12682470,18,1197502,11,0,f,2013-03-28
3,12996040,15,1197502,9,0,f,2013-03-25
4,13089312,15,1204821,9,0,f,2013-04-01


In [ ]:
tr.head()

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,7,707,1078778070,12564,2012-03-02,12.0,OZ,1,7.59
1,86246,205,63,6319,107654575,17876,2012-03-02,64.0,OZ,1,1.59
2,86246,205,97,9753,1022027929,0,2012-03-02,1.0,CT,1,5.99
3,86246,205,25,2509,107996777,31373,2012-03-02,16.0,OZ,1,1.99
4,86246,205,55,5555,107684070,32094,2012-03-02,16.0,OZ,2,10.38


In [ ]:
his.groupby(['id', 'chain']).size().reset_index(name='count')

,id,chain,count
0,86246,205,1
1,86252,205,1
2,12682470,18,1
3,12996040,15,1
4,13089312,15,1
...,...,...,...
160052,4712158291,214,1
160053,4715039820,166,1
160054,4797277036,152,1
160055,4804019740,153,1


In [ ]:
his_id=pd.merge(his,tr,how='inner',on=['id', 'chain'])

In [ ]:
his_id.head()

,id,chain,offer,market,repeattrips,repeater,offerdate,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,1208251,34,5,t,2013-04-24,7,707,1078778070,12564,2012-03-02,12.0,OZ,1,7.59
1,86246,205,1208251,34,5,t,2013-04-24,63,6319,107654575,17876,2012-03-02,64.0,OZ,1,1.59
2,86246,205,1208251,34,5,t,2013-04-24,97,9753,1022027929,0,2012-03-02,1.0,CT,1,5.99
3,86246,205,1208251,34,5,t,2013-04-24,25,2509,107996777,31373,2012-03-02,16.0,OZ,1,1.99
4,86246,205,1208251,34,5,t,2013-04-24,55,5555,107684070,32094,2012-03-02,16.0,OZ,2,10.38


In [ ]:
his_id.isnull().sum()

id                        0
chain                     0
offer                     0
market                    0
repeattrips               0
repeater                  0
offerdate                 0
dept                      0
category                  0
company                   0
brand                     0
date                      0
productsize               0
productmeasure      5649686
purchasequantity          0
purchaseamount            0
dtype: int64

In [ ]:
his_id_offer=pd.merge(his_id,df,how='inner',on=['offer','category', 'brand', 'company'])

In [ ]:
his_id_offer_gd=pd.get_dummies(his_id_offer)

In [ ]:
his_id_offer.head()

,id,chain,offer,market,repeattrips,offerdate,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount,quantity,offervalue,repeater
0,86252,205,1197502,34,16,2013-03-27,32,3203,106414464,13474,2012-06-07,5.0,OZ,1,2.99,1,0.75,t
1,86252,205,1197502,34,16,2013-03-27,32,3203,106414464,13474,2013-03-03,5.0,OZ,1,1.99,1,0.75,t
2,16797008,88,1197502,14,1,2013-04-17,32,3203,106414464,13474,2012-03-16,6.0,OZ,1,3.49,1,0.75,t
3,23412989,17,1197502,4,0,2013-04-23,32,3203,106414464,13474,2012-04-03,5.0,OZ,1,3.49,1,0.75,f
4,53024321,18,1197502,11,3,2013-04-18,32,3203,106414464,13474,2013-02-27,5.0,OZ,2,-1.41,1,0.75,t


In [ ]:
his_id_offer.drop(labels=['offerdate','date','offervalue'],axis=1,inplace=True)

In [ ]:
final=pd.get_dummies(his_id_offer)

In [ ]:
final.head()

,id,chain,offer,market,repeattrips,dept,category,company,brand,productsize,purchasequantity,purchaseamount,quantity,productmeasure_CT,productmeasure_OZ,repeater
0,86252,205,1197502,34,16,32,3203,106414464,13474,5.0,1,2.99,1,False,True,1
1,86252,205,1197502,34,16,32,3203,106414464,13474,5.0,1,1.99,1,False,True,1
2,16797008,88,1197502,14,1,32,3203,106414464,13474,6.0,1,3.49,1,False,True,1
3,23412989,17,1197502,4,0,32,3203,106414464,13474,5.0,1,3.49,1,False,True,1
4,53024321,18,1197502,11,3,32,3203,106414464,13474,5.0,2,-1.41,1,False,True,1


**모델 구성**

In [ ]:
X = final.iloc[:,0:15]
Y = final.iloc[:,15]

In [76]:
X=X.astype('float32')
Y=Y.astype('float32')

In [80]:
model = Sequential()
model.add(Dense(24,  input_dim=15, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelpath="/content/model"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델을 실행합니다.
history=model.fit(X, Y, epochs=200, batch_size=10,validation_split=0.25,callbacks=[early_stopping_callback,checkpointer])

Epoch 1/200
6228/6228 [==============================] - 14s 2ms/step - loss: 497647.9375 - accuracy: 0.5375 - val_loss: 427577.7188 - val_accuracy: 0.5887
Epoch 2/200
6228/6228 [==============================] - 12s 2ms/step - loss: 198749.8438 - accuracy: 0.5410 - val_loss: 556240.8125 - val_accuracy: 0.4013
Epoch 3/200
6228/6228 [==============================] - 13s 2ms/step - loss: 143225.6094 - accuracy: 0.5478 - val_loss: 142060.2500 - val_accuracy: 0.3658
Epoch 4/200
6228/6228 [==============================] - 13s 2ms/step - loss: 66394.0078 - accuracy: 0.5608 - val_loss: 2693.5730 - val_accuracy: 0.3128
Epoch 5/200
6228/6228 [==============================] - 13s 2ms/step - loss: 20.9572 - accuracy: 0.5890 - val_loss: 0.8015 - val_accuracy: 0.2328
Epoch 6/200
6228/6228 [==============================] - 13s 2ms/step - loss: 0.6771 - accuracy: 0.5894 - val_loss: 0.7979 - val_accuracy: 0.2328
Epoch 7/200
6228/6228 [==============================] - 12s 2ms/step - loss: 0.6771 -